In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests10 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000

skip_shi = False
refinement_test=True
adapt_c = True

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )


In [5]:
# Input arrays
alpha = [0.01, 0.05, 0.10, 0.15]
results = np.array([
    [[0.98, 0.02, 0.00], [0.92, 0.08, 0.00]],
    [[0.86, 0.14, 0.00], [0.66, 0.34, 0.00]],
    [[0.66, 0.34, 0.00], [0.50, 0.50, 0.00]],
    [[0.54, 0.46, 0.00], [0.41, 0.59, 0.00]]
])
# Column headers

def print_mc2(alpha_levels ,test_results ):
    table = "\\begin{tabular}{c|cccc}\n"
    table += "\\hline\n"
    table += "\\textbf{$\\alpha$} & {} & \\textbf{No selection} & \\textbf{Model 1} & \\textbf{Model 2} \\\\\n"
    table += "\\hline\n"

    for i in range(len(alpha_levels)):
        alpha = alpha_levels[i]
        table += "{$%.2f$} & \\textbf{Normal} & %.2f & %.2f & %.2f \\\\\n" % (alpha, test_results[i][0][0], test_results[i][0][1], test_results[i][0][2])
        table += "& \\textbf{Bootstrap-ND} & %.2f & %.2f & %.2f \\\\\n" % (test_results[i][1][0], test_results[i][1][1], test_results[i][1][2])
        table += "& \\textbf{Shi (2015)} & %.2f & %.2f & %.2f \\\\\n" % (test_results[i][2][0], test_results[i][2][1], test_results[i][2][2])
        table += "\\hline\n"

    table += "\\end{tabular}"

    print(table)

#print_mc2(alpha,results)

# evidence of power

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
import vuong_tests5

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)

res0 = vuong_tests5.monte_carlo(1,gen_data_ex,setup_shi_ex,trials=500,biascorrect=False)

nobs=250
a1,a2 = np.sqrt(1.09-1), 0.00
c1,c2 = calc_c(nobs)
print(c1)

cstar 64
64 cstar 25.83544889016606
64 cstar 25.83544889016606
64 cstar 25.83544889016606
0.0006324555320336759


# a  = .25, k= 4,n=500

In [8]:
num_params=4
nobs=500
a1,a2 = np.sqrt(1.09**.5-1), 0.00
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,
                                          data_tuned_c=data_tuned_c,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3,shi])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.58, 5th: 1.12, 10th: 1.54, 30th: 2.72, 50th: 3.95, 70th: 5.49, 90th: 8.30, 95th: 9.99, 99th: 14.43
bonus term 0.062071041994589564 // variance stats 0.2036133222559181 //part1 3.465724215775732 //part2 0.017909977289031414
-2.449567887627911 0.4938669514719498 2.3621178925767596
-2.57370997161709 0.4938669514719498 2.2379758085875805
---
Quantiles: 1th: 0.64, 5th: 1.22, 10th: 1.69, 30th: 2.95, 50th: 4.26, 70th: 5.87, 90th: 9.18, 95th: 10.98, 99th: 16.95
bonus term 0.055649322229545446 // variance stats 0.2514548220330849 //part1 3.465724215775732 //part2 0.016057054388873084
-2.4007714006465277 2.1148738099606845 2.7661456277446965
-2.512070045105619 2.1148738099606845 2.6548469832856054
---
Quantiles: 1th: 0.61, 5th: 1.18, 10th: 1.65, 30th: 3.08, 50th: 4.40, 70th: 6.19, 90th: 9.30, 95th: 11.13, 99th: 14.94
bonus term 0.05992889308094808 // variance stats 0.2363183658364669 //part1 3.465724215775732 //part2 0.017291881681801447
-2.3396715530077348 2.362809741070015 2.

Quantiles: 1th: 0.65, 5th: 1.21, 10th: 1.65, 30th: 3.00, 50th: 4.31, 70th: 5.94, 90th: 9.17, 95th: 10.76, 99th: 14.34
bonus term 0.054813710634406314 // variance stats 0.2551843149923896 //part1 3.465724215775732 //part2 0.015815947034936644
-1.8779056259772389 1.972477642759068 2.162331692080226
-1.9875330472460513 1.972477642759068 2.0527042708114136
---
Quantiles: 1th: 0.63, 5th: 1.18, 10th: 1.60, 30th: 2.98, 50th: 4.25, 70th: 6.00, 90th: 9.40, 95th: 11.21, 99th: 15.28
bonus term 0.065767634470171 // variance stats 0.21742726444306573 //part1 3.465724215775732 //part2 0.018976592012371143
-1.8230851196546614 2.457536232512825 2.1154221307894816
-1.9546203885950033 2.457536232512825 1.9838868618491392
---
Quantiles: 1th: 0.55, 5th: 1.16, 10th: 1.57, 30th: 2.91, 50th: 4.18, 70th: 5.86, 90th: 8.90, 95th: 10.52, 99th: 15.23
bonus term 0.06003806274968603 // variance stats 0.2256749059892549 //part1 3.465724215775732 //part2 0.01732338149596468
-2.0152252048635146 1.157730788917002 2.038

Quantiles: 1th: 0.62, 5th: 1.16, 10th: 1.66, 30th: 3.09, 50th: 4.55, 70th: 6.47, 90th: 10.23, 95th: 12.27, 99th: 17.64
bonus term 0.11977856426357975 // variance stats 0.12774894380574672 //part1 3.465724215775732 //part2 0.03456090467855352
-1.4630550376102835 2.010082450071292 1.749710078668365
-1.7026121661374432 2.010082450071292 1.5101529501412054
---
Quantiles: 1th: 0.62, 5th: 1.14, 10th: 1.59, 30th: 2.87, 50th: 4.17, 70th: 5.74, 90th: 8.84, 95th: 10.50, 99th: 14.77
bonus term 0.05484863853489948 // variance stats 0.2458586630768744 //part1 3.465724215775732 //part2 0.01582602513068765
-1.6028058207968594 3.128605971563354 1.7323767585040633
-1.7125030978666582 3.128605971563354 1.6226794814342644
---
Quantiles: 1th: 0.59, 5th: 1.21, 10th: 1.62, 30th: 2.93, 50th: 4.23, 70th: 5.89, 90th: 9.06, 95th: 10.79, 99th: 14.82
bonus term 0.05910183277991214 // variance stats 0.23358295573317986 //part1 3.465724215775732 //part2 0.01705324171810462
-1.5314752042070192 2.1475853065062167 1.5

Quantiles: 1th: 0.56, 5th: 1.00, 10th: 1.41, 30th: 2.64, 50th: 3.72, 70th: 5.20, 90th: 7.92, 95th: 9.37, 99th: 12.63
bonus term 0.06918175745205377 // variance stats 0.17394729669916778 //part1 3.465724215775732 //part2 0.019961702993314723
-1.370425643170088 1.4782813976510794 1.5659118886950778
-1.5087891580741954 1.4782813976510794 1.4275483737909704
---
Quantiles: 1th: 0.62, 5th: 1.24, 10th: 1.70, 30th: 3.12, 50th: 4.52, 70th: 6.32, 90th: 9.79, 95th: 11.73, 99th: 16.14
bonus term 0.07375269140163264 // variance stats 0.2014507417355565 //part1 3.465724215775732 //part2 0.021280600189107834
-1.3321990345215917 2.248623020049753 1.5279756201592707
-1.479704417324857 2.248623020049753 1.3804702373560054
---
Quantiles: 1th: 0.59, 5th: 1.14, 10th: 1.66, 30th: 2.87, 50th: 4.14, 70th: 5.76, 90th: 8.74, 95th: 10.40, 99th: 14.54
bonus term 0.056780206218123214 // variance stats 0.2346658031325825 //part1 3.465724215775732 //part2 0.016383359633655708
-1.3595713904935696 2.900138291721534 1.

# a  = .25, k= 4, n=250

In [9]:
num_params=4
nobs=250
a1,a2 = np.sqrt(1.09-1), 0.00
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,
                                          data_tuned_c=data_tuned_c,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3,shi])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.75, 5th: 1.38, 10th: 1.91, 30th: 3.47, 50th: 4.98, 70th: 7.03, 90th: 10.84, 95th: 13.13, 99th: 18.27
bonus term 0.059597683396900164 // variance stats 0.34028768021199796 //part1 3.017088168272582 //part2 0.019753378115901235
-2.48190788889625 1.19026666668622 2.3398390751358535
-2.60110325569005 1.19026666668622 2.2206437083420534
---
Quantiles: 1th: 0.60, 5th: 1.07, 10th: 1.41, 30th: 2.57, 50th: 3.69, 70th: 5.06, 90th: 7.76, 95th: 9.25, 99th: 12.56
bonus term 0.050049467985703804 // variance stats 0.2909293377776056 //part1 3.017088168272582 //part2 0.0165886660230945
-2.615729369466803 1.6520699844416875 2.496982557547979
-2.715828305438211 1.6520699844416875 2.396883621576571
---
Quantiles: 1th: 0.74, 5th: 1.37, 10th: 1.77, 30th: 3.17, 50th: 4.57, 70th: 6.38, 90th: 9.85, 95th: 11.98, 99th: 16.12
bonus term 0.05543241401952991 // variance stats 0.33288130609167277 //part1 3.017088168272582 //part2 0.018372818733788432
-2.613100914654328 2.7076993855556255 2.7006872

Quantiles: 1th: 0.50, 5th: 1.03, 10th: 1.41, 30th: 2.64, 50th: 3.81, 70th: 5.36, 90th: 8.46, 95th: 9.87, 99th: 13.85
bonus term 0.10183697528651038 // variance stats 0.1532357936722737 //part1 3.017088168272582 //part2 0.033753397185212725
-1.8376578527298362 1.5960260313042554 2.2833961369242233
-2.041331803302857 1.5960260313042554 2.0797221863512028
---
Quantiles: 1th: 0.68, 5th: 1.22, 10th: 1.69, 30th: 3.14, 50th: 4.55, 70th: 6.20, 90th: 9.54, 95th: 11.42, 99th: 15.98
bonus term 0.059039411940146884 // variance stats 0.302165810192197 //part1 3.017088168272582 //part2 0.019568341608641023
-1.8338647541025095 1.5907143823053447 1.9003000179906528
-1.9519435779828034 1.5907143823053447 1.782221194110359
---
Quantiles: 1th: 0.69, 5th: 1.30, 10th: 1.76, 30th: 3.08, 50th: 4.41, 70th: 6.15, 90th: 9.68, 95th: 11.64, 99th: 16.42
bonus term 0.06058786991245044 // variance stats 0.2987837220912652 //part1 3.017088168272582 //part2 0.02008157088333938
-1.8254089454440368 2.6322966962286194 2.

Quantiles: 1th: 0.81, 5th: 1.49, 10th: 2.05, 30th: 3.69, 50th: 5.34, 70th: 7.28, 90th: 11.34, 95th: 13.75, 99th: 19.36
bonus term 0.06601766832619041 // variance stats 0.3204659248878058 //part1 3.017088168272582 //part2 0.021881252599916058
-1.5265695648709667 1.6316386729274877 1.70661297697368
-1.6586049015233475 1.6316386729274877 1.5745776403212992
---
Quantiles: 1th: 0.75, 5th: 1.43, 10th: 1.87, 30th: 3.37, 50th: 4.97, 70th: 7.00, 90th: 11.20, 95th: 13.92, 99th: 20.15
bonus term 0.054658551424287105 // variance stats 0.38378566260029223 //part1 3.017088168272582 //part2 0.018116325535021262
-1.5913969011740499 2.2334682324055124 1.8972026929740944
-1.7007140040226238 2.2334682324055124 1.7878855901255204
---
Quantiles: 1th: 0.76, 5th: 1.46, 10th: 1.98, 30th: 3.53, 50th: 5.18, 70th: 7.29, 90th: 11.33, 95th: 13.64, 99th: 19.77
bonus term 0.053856919810270916 // variance stats 0.3942135001546401 //part1 3.017088168272582 //part2 0.017850628422670992
-1.5134836320832243 2.55845027613

Quantiles: 1th: 0.69, 5th: 1.26, 10th: 1.71, 30th: 3.08, 50th: 4.42, 70th: 6.12, 90th: 9.57, 95th: 11.67, 99th: 16.43
bonus term 0.05420272822879384 // variance stats 0.3309878358581807 //part1 3.017088168272582 //part2 0.01796524503287132
-1.4105685780686275 1.817958792042446 1.2929355280407793
-1.518974034526215 1.817958792042446 1.1845300715831917
---
Quantiles: 1th: 0.66, 5th: 1.22, 10th: 1.65, 30th: 2.91, 50th: 4.13, 70th: 5.76, 90th: 8.69, 95th: 10.36, 99th: 14.12
bonus term 0.047351704305142635 // variance stats 0.34459629111427603 //part1 3.017088168272582 //part2 0.015694504656207513
-1.4443199286917694 2.736483296063145 1.5931823750660492
-1.5390233373020545 2.736483296063145 1.498478966455764
---
Quantiles: 1th: 0.78, 5th: 1.43, 10th: 1.93, 30th: 3.43, 50th: 4.92, 70th: 6.78, 90th: 10.50, 95th: 12.89, 99th: 18.20
bonus term 0.0572151014475196 // variance stats 0.3434118408011073 //part1 3.017088168272582 //part2 0.01896368228452462
-1.2481851638789583 1.496421606803743 1.627

# size stuff

# a  = .25, k= 4,n=250

In [10]:
num_params=4
nobs=250
a=.25 
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,
                                          data_tuned_c=data_tuned_c,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3,shi])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.59, 5th: 1.17, 10th: 1.53, 30th: 2.69, 50th: 3.87, 70th: 5.34, 90th: 8.17, 95th: 9.87, 99th: 12.99
bonus term 0.04425634856926983 // variance stats 0.34722056451089467 //part1 3.017088168272582 //part2 0.014668563230821514
-2.4400686944497476 0.44322364493449534 2.483431241635008
-2.528581391588287 0.44322364493449534 2.3949185444964685
---
Quantiles: 1th: 0.68, 5th: 1.22, 10th: 1.66, 30th: 2.88, 50th: 4.13, 70th: 5.86, 90th: 9.26, 95th: 11.64, 99th: 16.43
bonus term 0.042884973384562416 // variance stats 0.40627669891482016 //part1 3.017088168272582 //part2 0.01421402723179813
-2.3581212044105673 2.1134469211689595 2.4755335200159485
-2.4438911511796917 2.1134469211689595 2.389763573246824
---
Quantiles: 1th: 0.82, 5th: 1.46, 10th: 1.91, 30th: 3.30, 50th: 4.67, 70th: 6.48, 90th: 9.86, 95th: 12.02, 99th: 16.23
bonus term 0.04186885699243317 // variance stats 0.44304187095237263 //part1 3.017088168272582 //part2 0.013877240126000352
-2.6430821363760852 0.37986096794470

Quantiles: 1th: 0.60, 5th: 1.08, 10th: 1.47, 30th: 2.62, 50th: 3.69, 70th: 5.08, 90th: 7.73, 95th: 9.12, 99th: 12.20
bonus term 0.048727204397505344 // variance stats 0.29767732563619387 //part1 3.017088168272582 //part2 0.016150407836905824
-1.961424401563155 -0.996775394573539 1.9558476224177426
-2.058878810358166 -0.996775394573539 1.858393213622732
---
Quantiles: 1th: 0.67, 5th: 1.21, 10th: 1.65, 30th: 2.90, 50th: 4.20, 70th: 5.78, 90th: 9.18, 95th: 11.05, 99th: 16.19
bonus term 0.04367742918415071 // variance stats 0.39515837514877056 //part1 3.017088168272582 //part2 0.014476683062649109
-1.731868658025583 0.5302270569735276 2.075683969948771
-1.8192235163938844 0.5302270569735276 1.9883291115804698
---
Quantiles: 1th: 0.66, 5th: 1.17, 10th: 1.62, 30th: 2.84, 50th: 4.07, 70th: 5.66, 90th: 8.88, 95th: 10.64, 99th: 14.96
bonus term 0.05875647072688065 // variance stats 0.2827346344886336 //part1 3.017088168272582 //part2 0.019474562044542888
-2.124675277730448 0.7451856456152047 2.

-1.6133651697989295 0.11417106620540401 1.6363143058146168
-1.7230266549860174 0.11417106620540401 1.526652820627529
---
Quantiles: 1th: 0.63, 5th: 1.12, 10th: 1.54, 30th: 2.73, 50th: 3.76, 70th: 5.29, 90th: 7.95, 95th: 9.61, 99th: 13.05
bonus term 0.0495537110192154 // variance stats 0.30107801777424786 //part1 3.017088168272582 //part2 0.016424349656174324
-1.5862284675742107 -0.6683471234553154 1.790590211257501
-1.6853358896126416 -0.6683471234553154 1.69148278921907
---
Quantiles: 1th: 0.79, 5th: 1.36, 10th: 1.84, 30th: 3.16, 50th: 4.56, 70th: 6.34, 90th: 9.58, 95th: 11.62, 99th: 15.81
bonus term 0.041831491066731195 // variance stats 0.4307701312535028 //part1 3.017088168272582 //part2 0.013864855361744895
-1.5204979686107147 0.3864860133767823 1.6325146646324373
-1.6041609507441772 0.3864860133767823 1.5488516824989749
---
Quantiles: 1th: 0.65, 5th: 1.29, 10th: 1.73, 30th: 3.03, 50th: 4.36, 70th: 5.98, 90th: 9.32, 95th: 11.30, 99th: 15.46
bonus term 0.050585100884815985 // varia

-1.4118849120318089 0.40359806696520706 1.3888539852595962
-1.480031796025103 0.40359806696520706 1.320707101266302
---
Quantiles: 1th: 0.76, 5th: 1.42, 10th: 1.90, 30th: 3.26, 50th: 4.58, 70th: 6.32, 90th: 9.61, 95th: 11.51, 99th: 15.24
bonus term 0.03783798277344895 // variance stats 0.4784762307824897 //part1 3.017088168272582 //part2 0.012541225401149908
-1.4178140255169465 1.4044238099245558 1.3892563376852394
-1.4934899910638444 1.4044238099245558 1.3135803721383414
---
Quantiles: 1th: 0.66, 5th: 1.28, 10th: 1.78, 30th: 3.24, 50th: 4.72, 70th: 6.54, 90th: 9.96, 95th: 11.97, 99th: 17.22
bonus term 0.07220182194574247 // variance stats 0.2570200946830219 //part1 3.017088168272582 //part2 0.02393096188073325
-1.1483778483061158 1.0481857402980617 1.544592599731852
-1.2927814921976009 1.0481857402980617 1.4001889558403668
---
Quantiles: 1th: 0.77, 5th: 1.51, 10th: 2.14, 30th: 3.77, 50th: 5.38, 70th: 7.59, 90th: 11.67, 95th: 14.17, 99th: 19.75
bonus term 0.04609271956840299 // varianc

# a  = .25, k= 4,n=500

In [11]:
###### num_params=4
nobs=500
a=.25 
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3,shi])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.78, 5th: 1.46, 10th: 1.97, 30th: 3.45, 50th: 4.78, 70th: 6.52, 90th: 9.84, 95th: 11.72, 99th: 15.76
bonus term 0.004518564702892487 // variance stats 0.34256413834025545 //part1 3.465724215775732 //part2 0.0013037865743397296
-2.4671602278117493 0.6281679632890167 2.56199071023512
-2.476197357217534 0.6281679632890167 2.552953580829335
---
Quantiles: 1th: 0.74, 5th: 1.48, 10th: 2.05, 30th: 3.70, 50th: 5.28, 70th: 7.39, 90th: 11.25, 95th: 13.49, 99th: 18.52
bonus term 0.004637670484324613 // variance stats 0.33375483736674144 //part1 3.465724215775732 //part2 0.0013381533542727562
-2.543507026699736 -0.9217443939932409 2.6504147398426494
-2.5527823676683856 -0.9217443939932409 2.641139398874
---
Quantiles: 1th: 0.71, 5th: 1.25, 10th: 1.66, 30th: 2.94, 50th: 4.23, 70th: 5.77, 90th: 8.91, 95th: 10.68, 99th: 14.45
bonus term 0.004471784769074449 // variance stats 0.34615242560608284 //part1 3.465724215775732 //part2 0.0012902886931162034
-2.6301585899760265 -1.93755158371

-2.0147494528281205 0.4296090898861369 2.1881723667712563
-2.022215475545391 0.4296090898861369 2.1807063440539856
---
Quantiles: 1th: 0.65, 5th: 1.25, 10th: 1.68, 30th: 2.88, 50th: 4.04, 70th: 5.62, 90th: 8.68, 95th: 10.50, 99th: 14.50
bonus term 0.004054658433945635 // variance stats 0.3818091497450967 //part1 3.465724215775732 //part2 0.0011699310682278512
-1.9199447677353085 -0.6674283168685111 1.7852578990307106
-1.9280540846031997 -0.6674283168685111 1.7771485821628195
---
Quantiles: 1th: 0.72, 5th: 1.30, 10th: 1.72, 30th: 3.06, 50th: 4.36, 70th: 6.06, 90th: 9.07, 95th: 10.90, 99th: 15.07
bonus term 0.005136092970926987 // variance stats 0.30132282762487933 //part1 3.465724215775732 //part2 0.001481968169177413
-2.0619979161502187 1.6458145119934477 2.1064547241383083
-2.072270102092073 1.6458145119934477 2.096182538196454
---
Quantiles: 1th: 0.73, 5th: 1.32, 10th: 1.80, 30th: 3.25, 50th: 4.68, 70th: 6.47, 90th: 9.85, 95th: 11.81, 99th: 16.32
bonus term 0.0048813221869012475 // v

-1.7761413420359997 1.8068337528086094 1.5876782923926271
-1.7828128074665306 1.8068337528086094 1.5810068269620963
---
Quantiles: 1th: 0.72, 5th: 1.36, 10th: 1.77, 30th: 3.19, 50th: 4.51, 70th: 6.31, 90th: 9.60, 95th: 11.59, 99th: 16.06
bonus term 0.003902891213372716 // variance stats 0.39667351122453975 //part1 3.465724215775732 //part2 0.0011261401572597815
-1.6012330927942842 0.6436810633292479 1.6300110940216281
-1.6090388752210296 0.6436810633292479 1.6222053115948827
---
Quantiles: 1th: 0.58, 5th: 1.20, 10th: 1.64, 30th: 2.87, 50th: 4.18, 70th: 5.71, 90th: 8.69, 95th: 10.56, 99th: 14.13
bonus term 0.005539421015768988 // variance stats 0.27935079835127313 //part1 3.465724215775732 //part2 0.001598344435646072
-1.5823828027055449 -0.5821409086064091 1.5949092705902164
-1.593461644737083 -0.5821409086064091 1.5838304285586782
---
Quantiles: 1th: 0.64, 5th: 1.15, 10th: 1.58, 30th: 2.79, 50th: 3.86, 70th: 5.26, 90th: 8.06, 95th: 9.57, 99th: 13.14
bonus term 0.004947332612352619 // 

-1.4025748400822697 -0.3677712750511521 1.4734896815233012
-1.410784451071912 -0.3677712750511521 1.4652800705336588
---
Quantiles: 1th: 0.75, 5th: 1.42, 10th: 1.92, 30th: 3.35, 50th: 4.76, 70th: 6.67, 90th: 10.16, 95th: 12.23, 99th: 16.45
bonus term 0.0036183567478742934 // variance stats 0.42790164627269334 //part1 3.465724215775732 //part2 0.0010440405879393946
-1.4413170130103075 -0.3479735499808174 1.364430177240547
-1.448553726506056 -0.3479735499808174 1.3571934637447984
---
Quantiles: 1th: 0.70, 5th: 1.32, 10th: 1.76, 30th: 3.12, 50th: 4.37, 70th: 6.01, 90th: 9.43, 95th: 11.43, 99th: 16.68
bonus term 0.003823245720607664 // variance stats 0.40494629255403264 //part1 3.465724215775732 //part2 0.0011031592482761666
-1.4583454443754709 -0.4274931277854294 1.4969480888848137
-1.4659919358166864 -0.4274931277854294 1.4893015974435981
---
Quantiles: 1th: 0.80, 5th: 1.39, 10th: 1.89, 30th: 3.55, 50th: 5.11, 70th: 7.04, 90th: 11.04, 95th: 13.27, 99th: 17.95
bonus term 0.005412426012769